In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars file:///home/kiq/Downloads/postgresql-42.4.0.jar pyspark-shell'

In [22]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import functions as Func

In [5]:
sc = SparkContext(appName="Postgres")
spark = SparkSession.builder\
        .master('local')\
        .appName("Postgres")\
        .getOrCreate()

22/06/15 23:03:00 WARN Utils: Your hostname, base resolves to a loopback address: 127.0.1.1; using 192.168.0.8 instead (on interface wlo1)
22/06/15 23:03:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/06/15 23:03:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [11]:
df_vendas = spark.read.format('jdbc')\
    .option("url","jdbc:postgresql://localhost:5432/vendas")\
    .option('dbtable','vendas')\
    .option('user','postgres')\
    .option('password','kaka2014')\
    .option('driver','org.postgresql.Driver')\
    .load()

In [12]:
df_vendas.show()

+-------+----------+---------+----------+--------+
|idvenda|idvendedor|idcliente|      data|   total|
+-------+----------+---------+----------+--------+
|      1|         1|        1|2016-01-01| 8053.60|
|      2|         6|      185|2016-01-01|  150.40|
|      3|         7|       31|2016-01-02| 6087.00|
|      4|         5|        1|2016-01-02|13828.60|
|      5|         8|      120|2016-01-03|26096.66|
|      6|         9|       74|2016-01-04|18402.00|
|      7|         9|      191|2016-01-06| 7524.20|
|      8|         6|        2|2016-01-06|12036.60|
|      9|         7|       91|2016-01-06| 2804.75|
|     10|         2|      202|2016-01-06| 8852.00|
|     11|         1|        3|2016-01-08|16545.25|
|     12|         7|       32|2016-01-09|11411.88|
|     13|         4|      136|2016-01-10|15829.70|
|     14|         3|      249|2016-01-12| 6154.36|
|     15|         4|        5|2016-01-12| 3255.08|
|     16|         7|      192|2016-01-13| 2901.25|
|     17|         2|       79|2

In [8]:
df_clientes = spark.read.format('jdbc')\
    .option("url","jdbc:postgresql://localhost:5432/vendas")\
    .option('dbtable','clientes')\
    .option('user','postgres')\
    .option('password','kaka2014')\
    .option('driver','org.postgresql.Driver')\
    .load()

In [9]:
df_clientes.show()

+---------+--------------------+------+----+--------+
|idcliente|             cliente|estado|sexo|  status|
+---------+--------------------+------+----+--------+
|        1|Adelina Buenaventura|    RJ|   M|  Silver|
|        2|        Adelino Gago|    RJ|   M|  Silver|
|        3|       Adélio Lisboa|    SE|   M|  Silver|
|        4|       Adérito Bahía|    MA|   M|  Silver|
|        5|     Adolfo Patrício|    PE|   M|  Silver|
|        6|    Adriana Guedelha|    RO|   F|Platinum|
|        7|       Aida Dorneles|    RN|   F|  Silver|
|        8|   Alarico Quinterno|    AC|   M|  Silver|
|        9|    Alberto Cezimbra|    AM|   M|  Silver|
|       10|    Alberto Monsanto|    RN|   M|    Gold|
|       11|       Albino Canela|    AC|   M|  Silver|
|       12|     Alceste Varanda|    RR|   F|  Silver|
|       13|  Alcides Carvalhais|    RO|   M|  Silver|
|       14|        Aldo Martins|    GO|   M|  Silver|
|       15|   Alexandra Tabares|    MG|   F|  Silver|
|       16|      Alfredo Cot

In [24]:
df_vendas.schema

StructType(List(StructField(idvenda,IntegerType,true),StructField(idvendedor,IntegerType,true),StructField(idcliente,IntegerType,true),StructField(data,DateType,true),StructField(total,DecimalType(10,2),true)))

In [42]:
df_datavenda = df_vendas.groupBy('data').agg(Func.sum("total").alias("total"))
df_datavenda.show()

+----------+--------+
|      data|   total|
+----------+--------+
|2016-04-25|16184.67|
|2016-05-03| 8514.01|
|2016-08-15|22880.88|
|2016-08-31|15346.92|
|2016-07-17| 3893.25|
|2016-12-19|13439.78|
|2016-07-03| 9446.44|
|2016-08-23|19438.51|
|2016-05-26| 9122.82|
|2016-06-02|28660.30|
|2016-01-13|18730.95|
|2016-01-01| 8204.00|
|2016-04-22| 5180.60|
|2016-06-16| 6737.03|
|2016-01-19|11290.19|
|2016-05-09| 7652.35|
|2016-07-19|14416.40|
|2016-09-15| 5932.00|
|2016-02-08| 5865.00|
|2016-10-07| 3131.90|
+----------+--------+
only showing top 20 rows



In [44]:
df_datavenda.write.format('jdbc')\
    .option("url","jdbc:postgresql://localhost:5432/vendas")\
    .option('dbtable','datavenda')\
    .option('user','postgres')\
    .option('password','kaka2014')\
    .option('driver','org.postgresql.Driver')\
    .save()